In [1]:
import os
import cv2
import csv
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

In [8]:
SRC = '/Users/fathe/OneDrive/Documents/UK/MFF/Thesis/input'
LANDSCAPE = 0
VIDEO = 1
SCALE = 6
scales = [6]

In [6]:
videos = [file for file in os.listdir(SRC) if file.endswith("MP4") and file.startswith("GX")]

In [7]:
videos

In [9]:
feature_params = dict(maxCorners=100,
                      qualityLevel=0.1,
                      minDistance=7,
                      blockSize=7)

lk_params = dict(winSize=(50, 50),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

err_threshold = 9

In [ ]:
def process():
    corners_not_found = []
    no_good_found = []
    rows = []
    moves = []
    vidcap = cv2.VideoCapture(os.path.join(SRC, videos[VIDEO]))
    width = int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH)/SCALE)
    height = int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT)/SCALE)
    pbar = tqdm(total=vidcap.get(cv2.CAP_PROP_FRAME_COUNT), desc=f"Extraction motion direction")

    _, old_frame = vidcap.read()
    old_gray = cv2.cvtColor(cv2.resize(old_frame, (width, height)), cv2.COLOR_BGR2GRAY)

    rows.append([vidcap.get(cv2.CAP_PROP_POS_FRAMES), 0, 0])

    while True:
        ret, frame = vidcap.read()
        if not ret:
            break
        corners = cv2.goodFeaturesToTrack(old_gray, **feature_params)
        frame_gray = cv2.cvtColor(cv2.resize(frame, (width, height)), cv2.COLOR_BGR2GRAY)

        if corners is not None:
            p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, corners, None, **lk_params)

            st = (st == 1) & (err < err_threshold)
            p1 = p1[st == 1]
            p0 = corners[st == 1]

            if p1.shape[0] > 0:
                move = np.mean(p1-p0, axis=0)
                moves.append(move)
                rows.append([vidcap.get(cv2.CAP_PROP_POS_FRAMES), rows[-1][1]+move[0], rows[-1][2]+move[1]])
            else:
                no_good_found.append(vidcap.get(cv2.CAP_PROP_POS_FRAMES))
                rows.append([vidcap.get(cv2.CAP_PROP_POS_FRAMES), rows[-1][1]+rows[-1][1]-rows[-2][1], rows[-1][2]+rows[-1][2]-rows[-2][2]])
        else:
            corners_not_found.append(vidcap.get(cv2.CAP_PROP_POS_FRAMES))
            rows.append([vidcap.get(cv2.CAP_PROP_POS_FRAMES), rows[-1][1]+rows[-1][1]-rows[-2][1], rows[-1][2]+rows[-1][2]-rows[-2][2]])

        old_gray = frame_gray

        pbar.update(1)

    total_time = pbar.last_print_t - pbar.start_t
    pbar.close()
    return rows, moves, total_time, corners_not_found, no_good_found

In [10]:
def process_with_intervals(intervals_horiz, intervals_vert, scale):
    corners_not_found = []
    no_good_found = []
    rows = []
    moves = []
    vidcap = cv2.VideoCapture(os.path.join(SRC, videos[VIDEO]))
    width = int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH)/scale)
    height = int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT)/scale)
    pbar = tqdm(total=vidcap.get(cv2.CAP_PROP_FRAME_COUNT), desc=f"Extraction motion direction for scale {scale}")
    
    _, old_frame = vidcap.read()
    old_gray = cv2.cvtColor(cv2.resize(old_frame, (width, height)), cv2.COLOR_BGR2GRAY)
    
    rows.append([vidcap.get(cv2.CAP_PROP_POS_FRAMES), 0, 0])
    
    start_horiz, end_horiz = intervals_horiz.pop(0)
    start_vert, end_vert = intervals_vert.pop(0)
    moving_horiz = False
    moving_vert = False
    
    while True:
        if vidcap.get(cv2.CAP_PROP_POS_FRAMES) == start_horiz:
            moving_horiz = True
        elif vidcap.get(cv2.CAP_PROP_POS_FRAMES) == end_horiz:
            moving_horiz = False
            if len(intervals_horiz)>0:
                start_horiz, end_horiz = intervals_horiz.pop(0)
            
        if vidcap.get(cv2.CAP_PROP_POS_FRAMES) == start_vert:
            moving_vert = True
        elif vidcap.get(cv2.CAP_PROP_POS_FRAMES) == end_vert:
            moving_vert = False
            if len(intervals_vert)>0:
                start_vert, end_vert = intervals_vert.pop(0)   
            
        ret, frame = vidcap.read()
        if not ret:
            break
        corners = cv2.goodFeaturesToTrack(old_gray, **feature_params)
        frame_gray = cv2.cvtColor(cv2.resize(frame, (width, height)), cv2.COLOR_BGR2GRAY)
    
        if corners is not None:
            p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, corners, None, **lk_params)
        
            st = (st == 1) & (err < err_threshold)
            p1 = p1[st == 1]
            p0 = corners[st == 1]
        
            if p1.shape[0] > 0:
                move = np.mean(p1-p0, axis=0)
                moves.append(move)
                if moving_horiz and LANDSCAPE == 0:
                    rows.append([vidcap.get(cv2.CAP_PROP_POS_FRAMES), rows[-1][1], rows[-1][2]+move[1]])
                elif moving_vert and LANDSCAPE == 0:
                    rows.append([vidcap.get(cv2.CAP_PROP_POS_FRAMES), rows[-1][1]+move[0], rows[-1][2]])
                elif moving_horiz and LANDSCAPE == 1:
                    rows.append([vidcap.get(cv2.CAP_PROP_POS_FRAMES), rows[-1][1]+move[0], rows[-1][2]])
                elif moving_vert and LANDSCAPE == 1:
                    rows.append([vidcap.get(cv2.CAP_PROP_POS_FRAMES), rows[-1][1], rows[-1][2]+move[1]])
                else:
                    rows.append([vidcap.get(cv2.CAP_PROP_POS_FRAMES), rows[-1][1], rows[-1][2]])
            else:
                no_good_found.append(vidcap.get(cv2.CAP_PROP_POS_FRAMES))
                rows.append([vidcap.get(cv2.CAP_PROP_POS_FRAMES), rows[-1][1]+rows[-1][1]-rows[-2][1], rows[-1][2]+rows[-1][2]-rows[-2][2]])
        else:
            corners_not_found.append(vidcap.get(cv2.CAP_PROP_POS_FRAMES))
            rows.append([vidcap.get(cv2.CAP_PROP_POS_FRAMES), rows[-1][1]+rows[-1][1]-rows[-2][1], rows[-1][2]+rows[-1][2]-rows[-2][2]])
        
        old_gray = frame_gray
    
        pbar.update(1)
        
    total_time = pbar.last_print_t - pbar.start_t
    pbar.close()
    return rows, moves, total_time, corners_not_found, no_good_found

In [52]:
# rows, _, total_time, corners_not_found, no_good_found = process_with_intervals(intervals_horizontal, intervals_vertical)

In [53]:
# corners_not_found, no_good_found, total_time

In [54]:
# fields = ["frame_ID", "x_shift", "y_shift"]
# metadata = [
#     VIDEO,
#     SCALE,
#     total_time,
#     len(corners_not_found),
#     len(no_good_found),
#     LANDSCAPE
# ]
# metadata_file = os.path.join(SRC, videos[VIDEO]).replace(".MP4", "-" + str(SCALE) + "-" + str(feature_params["qualityLevel"]) + "-intervals.npy")
# np.save(metadata_file, metadata)
# csv_file = os.path.join(SRC, videos[VIDEO]).replace(".MP4", "-" + str(SCALE) + "-" + str(feature_params["qualityLevel"]) + "-intervals.csv")
# with open(csv_file, 'w') as csvfile:
#     csvwriter = csv.writer(csvfile)
#     csvwriter.writerow(fields)
#     csvwriter.writerows(rows)

In [12]:
for scale in scales:
    intervals_horizontal = np.load(os.path.join(SRC, videos[VIDEO]).replace(".MP4", "-intervals_horizontal.npy")).tolist()
    intervals_vertical = np.load(os.path.join(SRC, videos[VIDEO]).replace(".MP4", "-intervals_vertical.npy")).tolist()
    rows, _, total_time, corners_not_found, no_good_found = process_with_intervals(intervals_horizontal, intervals_vertical, scale)
    
    fields = ["frame_ID", "x_shift", "y_shift"]
    metadata = [
        VIDEO,
        scale,
        total_time,
        len(corners_not_found),
        len(no_good_found),
        LANDSCAPE,
        feature_params["qualityLevel"],
        feature_params["minDistance"],
        feature_params["blockSize"],
        err_threshold
    ]
    metadata_file = os.path.join(SRC, videos[VIDEO]).replace(".MP4", "-" + str(scale) + "-" + str(feature_params["qualityLevel"]) + "-" +  str(feature_params["minDistance"]) + "-" +  str(feature_params["blockSize"]) + "-" + str(err_threshold) + "-" + str(lk_params['winSize']) + "-intervals-errfilter.npy")
    np.save(metadata_file, metadata)
    csv_file = os.path.join(SRC, videos[VIDEO]).replace(".MP4", "-" + str(scale) + "-" + str(feature_params["qualityLevel"]) + "-" +  str(feature_params["minDistance"]) + "-" +  str(feature_params["blockSize"]) + "-" + str(err_threshold) + "-" + str(lk_params['winSize']) + "-intervals-errfilter.csv")
    with open(csv_file, 'w') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(fields)
        csvwriter.writerows(rows)